## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.82722205, -0.02274594, -0.56141454,  0.647944  ],
        [ 0.1710548 ,  0.96194696,  0.21306878, -0.16151415],
        [ 0.5352045 , -0.27228785,  0.7996346 ,  0.09050262],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


In [8]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))

In [9]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [10]:
 gscene.NAME_DICT["floor"].dims = (2,1,0.01)

In [11]:
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.1)

## add sweep face

In [12]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [13]:
gscene.update_markers_all()

## add wp

In [14]:
TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.14
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [15]:
gscene.update_markers_all()

## add indy tool

In [16]:
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.07,0.07, 0.04), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts_col", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.11,0.11, 0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.03,0.03, 0.030), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.07,0.07, 0.030), rpy=(0,0,0), color=(0.0,0.8,0.0,0.2),
                            collision=True, fixed=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="indy0_brushbase", link_name="indy0_tcp",
                            center=(0,0,0.0775), dims=(0.06, 0.14, 0.015), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp",
                            center=(0,0,0.095), dims=(0.05,0.13, 0.015), rpy=(0,0,0), color=(1.0,1.0,0.94,1),
                            collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp",
                            center=(0,0,0.080), dims=(0.08,0.18, 0.02), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True, fixed=True)

In [17]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
                            center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True, fixed=True)

## add box

In [18]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [19]:
# give clearance from the track surface
s_builder.give_clearance(track, gtem_dict.values())

In [20]:
gscene.update_markers_all()

## Register binders

In [21]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [22]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)

## add objects

In [23]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [24]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [25]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [26]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [27]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [28]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [rcheck, gcheck, tcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [29]:
mplan.motion_filters = checkers_all

In [30]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [31]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

from pkg.utils.code_scraps import use_current_place_point_only
# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

('track_face', 'track_face', 0, 0, 0)


In [32]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [33]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [34]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [35]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
mplan.reset_log(False)
gtimer.reset()

In [42]:
obj_num = 2
sweep_num = len(sweep_list)
first_run = True
from_state = initial_state
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    for i_s in range(obj_num):
        obj_goal = ["track_face"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan")
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=200, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE,
                  wait_proc=first_run)
    gtimer.toc("plan")
    if not first_run:
        ppline.play_schedule(snode_schedule, period=0.01)
    ppline.wait_procs()
    time.sleep(0.5)
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    from_state = snode_schedule[-1].state
    first_run = False
ppline.play_schedule(snode_schedule, period=0.01)

Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try transition motion
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'tr

try: 8 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
branching: 4->9 (1.02/200.0 s, steps/err: 70(159.659147263 ms)/1.17179723376e-15)
try: 9 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
transition motion tried: True
result: 8 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = fail
try joint motion
result: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = success
branching: 1->10 (1.04/200.0 s, steps/err: 37(122.964143753 ms)/0.00120638422118)
try: 10 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
result: 9 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = fail
branching: 0->11 (1.05/200.0 s, steps/err: 44(373.979091644 ms)/0.00128952179785)
try: 10 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)

try joint motion
result: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 17 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0) = success
result: 19 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try transition motion
try: 20 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
branching: 17->20 (1.51/200.0 s, steps/err: 32(157.963991165 ms)/0.00205686350206)
result: 19

result: 26 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
result: 16 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = success
branching: 19->30 (1.92/200.0 s, steps/err: 77(423.074960709 ms)/0.00188161505534)
======================= terminated 3: first answer acquired from other agent ===============================
try: 27 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
======================= terminated 5: first answer acquired ===============================
======================= terminated 0: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired ===============================
try transition motion
branching: 16->31 (1.93/200.0 s, steps/err: 52(608.690977097 ms)/0.00152052731526)
++ adding return motion to acquired answer ++
try joint motion
======================= terminated 1: first answer acquired from other ag

result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 16 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
branching: 16->41 (2.66/200.0 s, steps/err: 69(927.804946899 ms)/0.00154774313329)
======================= terminated 2: first answer acquired ===============================
result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
======================= terminated 7: first answer acquired from other agent ===============================
try: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try transition

...

result: 17 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
result: 18 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 10 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try transition motion
try: 9 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
try: 8 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
try: 3 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
result: 8 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try transition motion
result: 10 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 16 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 6 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)


try joint motion
branching: 3->26 (2.05/200.0 s, steps/err: 69(736.341953278 ms)/0.00176693708698)
try transition motion
try: 26 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try: 15 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 25 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = fail
try: 26 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
result: 26 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = fail
try: 1 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
try: 16 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 5 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
transition motion tried: True
result: 16 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try joint mo

try joint motion
result: 17 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
try joint motion
try: 36 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
try: 28 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 35 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
constrained motion tried: True
result: 28 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = success
branching: 28->38 (2.49/200.0 s, steps/err: 38(285.177946091 ms)/0.00175642459972)
transition motion tried: True
result: 28 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 6 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
result: 36 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 38 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2)
try: 12 - ('track_face', 't

result: 46 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
result: 35 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
======================= terminated 10: first answer acquired from other agent ===============================
result: 33 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 35->49 (2.98/200.0 s, steps/err: 40(508.526086807 ms)/6.98553508762e-16)
======================= terminated 16: first answer acquired from other agent ===============================
try transition motion
constrained motion tried: True
branching: 33->50 (2.99/200.0 s, steps/err: 63(553.533792496 ms)/1.09227753469e-15)
result: 38 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2) = success
======================= terminated 1: first answer acquired from other agent ===============================
branching: 38->51 (3.01/200.0 s, steps/err: 28(454.686880112 ms)/0.001679

transition motion tried: True
result: 8 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 8->63 (3.3/200.0 s, steps/err: 75(1142.1148777 ms)/0.00187143631375)
======================= terminated 9: first answer acquired from other agent ===============================
joint motion tried: True
transition motion tried: True
======================= terminated 14: first answer acquired ===============================
result: 10 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
branching: 10->65 (3.34/200.0 s, steps/err: 128(1123.62098694 ms)/0.00167423251668)
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired ===============================
try joint motion
joint motion tried: True
======================= terminated 3: first answer acquired ===============================
transition motion tried:

In [43]:
print(gtimer)

plan: 	3371.0 ms/3 = 1123.528 ms (647.928/1935.223)



## Plan & execute

In [36]:
crob.reset_connection(False,True)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose)

connection command:
indy0: False
panda1: True


In [37]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
mplan.reset_log(False)
gtimer.reset()
obj_num = 2
sweep_num = len(sweep_list)
first_run = True
from_state = initial_state
snode_scedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan")
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=200, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE,
                  wait_proc=first_run)
    gtimer.toc("plan")
    if not first_run:
        snode_scedule_all.append(snode_schedule)
        ppline.execute_schedule(snode_schedule, vel_scale=0.5, acc_scale=1)
    ppline.wait_procs()
    time.sleep(0.5)
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    from_state = snode_schedule[-1].state
    first_run = False
snode_scedule_all.append(snode_schedule)
ppline.execute_schedule(snode_schedule, vel_scale=0.5, acc_scale=1)

Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_f

result: 3 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try joint motion
try: 6 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try transition motion
try: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 3 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = fail
try transition motion
result: 3 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
joint motion tried: True
try: 2 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face

joint motion tried: True
result: 14 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
result: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
result: 10 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0) = success
branching: 9->16 (1.48/200.0 s, steps/err: 63(230.260848999 ms)/1.09299106214e-15)
branching: 10->15 (1.47/200.0 s, steps/err: 29(193.146944046 ms)/0.0013496129024)
result: 14 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
result: 14 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0) = fail
try: 16 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
++ adding return motion to acquired answer ++
try: 14 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
try: 15 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 15 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_fa

constrained motion tried: True
======================= terminated 13: first answer acquired from other agent ===============================
result: 21 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0) = success
constrained motion tried: True
======================= terminated 2: first answer acquired from other agent ===============================
transition motion tried: True
++ adding return motion to acquired answer ++
result: 17 - ('track_face', 'track_face', 1, 0, 0)->('track_face', 'track_face', 2, 0, 0) = success
transition motion tried: True
result: 18 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = success
try joint motion
branching: 21->26 (1.94/200.0 s, steps/err: 34(189.339876175 ms)/0.00189608359903)
try joint motion
branching: 18->27 (1.95/200.0 s, steps/err: 53(151.406049728 ms)/0.00145732740755)
branching: 17->28 (1.96/200.0 s, steps/err: 38(359.208106995 ms)/0.00142844154446)
result: 15 - ('track_face', 'grip1', 2, 0, 0)

result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
branching: 0->40 (2.68/200.0 s, steps/err: 79(1257.77697563 ms)/0.0016290682999)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 1, 0)
try joint motion
======================= terminated 0: first answer acquired from other agent ===============================
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
result: 1 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
try: 1 - ('track_face', 'track_

...

++ adding return motion to acquired answer ++
======================= terminated 6: first answer acquired ===============================
try joint motion
======================= terminated 2: first answer acquired from other agent ===============================
constrained motion tried: True
result: 49 - ('grip1', 'track_face', 2, 1, 0)->('grip1', 'track_face', 2, 2, 0) = success
branching: 49->59 (4.47/200.0 s, steps/err: 33(697.244882584 ms)/0.000867712260128)
joint motion tried: True
++ adding return motion to acquired answer ++
try joint motion
transition motion tried: True
result: 2 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = success
branching: 2->61 (4.61/200.0 s, steps/err: 73(1100.94618797 ms)/0.00126876941073)
transition motion tried: True
joint motion tried: True
======================= terminated 11: first answer acquired from other agent ===============================
result: 51 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip

======================= terminated 14: first answer acquired from other agent ===============================
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: None
binder: None
binder: None
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
Use 20/20 agents
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try transition motion
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2

try: 2 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
result: 6 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
branching: 4->9 (0.99/200.0 s, steps/err: 59(244.961977005 ms)/1.03221868828e-15)
result: 2 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
result: 2 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
try: 9 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
transition motion tried: True
try: 7 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
transition motion tried: True
result: 7 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
try transition motion
try: 10 - ('track_face', 'track_face', 2, 2, 1)->('track_face', 'track_face', 2, 2, 2)
try: 6 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
try: 9 - ('track_face', 'track_face', 2, 2, 0)->('track_

try joint motion
======================= terminated 15: first answer acquired from other agent ===============================
joint motion tried: True
result: 7 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
branching: 7->20 (1.67/200.0 s, steps/err: 157(823.674917221 ms)/5.83577711451e-16)
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 17: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
joint motion tried: True
joint motion tried: True
======================= terminated 19: first answer acquired from other agent ===============================
joint motion tried: True
constrained motion tried: True
result: 14

binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0


In [38]:
print(gtimer)

plan: 	3372.0 ms/3 = 1124.031 ms (754.179/1819.838)

